<a href="https://colab.research.google.com/github/HARDIK218/Large-Language-Model/blob/main/jobScraper_AI_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -qqq install pip --progress-bar off
!pip -qqq install 'crewai[tools]'==0.28.8 --progress-bar off
!pip -qqq install langchain-groq==0.1.3 --progress-bar off
!pip -qqq install duckduckgo-search==5.3.0 --progress-bar off
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
!pip install --q -U duckduckgo-search
!pip install --q langchain_google_genai

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 kB 899.2 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 10.9 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of langchain to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 53.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 867.6/867.6 kB 45.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 45.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 814.5/814.5 kB 48.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 812.8/812.8 kB 53.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 810.5/810.5 kB 53.1 MB/s eta 0:00:00
  Attempting uninstall: langchain_community
    Found existing i

In [ ]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from crewai import Agent, Task, Crew
from langchain_google_genai import ChatGoogleGenerativeAI

In [ ]:
import os
import re
from datetime import datetime

import pandas as pd
import requests
from crewai import Agent, Crew, Process,Task
from crewai_tools import tool
from google.colab import userdata
from langchain.agents import load_tools
from langchain_community.tools import DuckDuckGoSearchResults
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq

def format_response (response: str) -> str:
    entries = re.split(r" (?<=]), (?=\[)", response)
    return [entry.strip("[]") for entry in entries]

os.environ ["GROQ_API_KEY"] = userdata.get("GROQ_API_KEY")

In [ ]:
llm = ChatGroq(temperature = 0.2,model_name="llama3-70b-8192")

In [ ]:
from langchain.tools import DuckDuckGoSearchRun
search_tool = DuckDuckGoSearchRun()
from crewai_tools import ScrapeWebsiteTool
scrape_tool = ScrapeWebsiteTool()

In [ ]:
job_analyst_agent = Agent(
    role="Job Analyst",
    goal="Monitor and analyze market data in real-time for the latest job opportunities in the market",
    backstory="Specializing in tech industry, this agent "
              "uses diffrent platforms like http://linkedin.com, https://www.naukri.com/, http://timesjobs.com and many more job posting site to analyze openings in the tech industry",
    verbose=True,
    allow_delegation=True,
    max_iter = 10,
    llm = llm,
    tools = [scrape_tool, search_tool]
)

In [ ]:
job_analysis_task = Task(
    description=(
        "Continuously monitor and analyze market data for "
        "the selected job type ({job_selection}) in the selected region({region}).using the provided tools to search and scrape the data from diffrent job posting sites "
    ),
    expected_output=(
        "a structured format of the data collected by tool giving the name of the company ,job title and any important information provided. "
    ),
    agent=job_analyst_agent,
)

In [ ]:
crew = Crew(
    agents=[job_analyst_agent],

    tasks=[job_analysis_task],

    verbose=2,
	#memory=True
)

In [ ]:
inputs = {
    "job_selection": "tech role",
    "region":"india",
}

result = crew.kickoff(inputs=inputs)

 [DEBUG]: == Working Agent: Job Analyst
 [INFO]: == Starting Task: Continuously monitor and analyze market data for the selected job type (tech role) in the selected region(india).using the provided tools to search and scrape the data from diffrent job posting sites 


> Entering new CrewAgentExecutor chain...
Thought: I need to monitor and analyze market data for tech roles in India.

Action: Read website content
Action Input: {"website_url": "https://www.linkedin.com/jobs/tech-jobs/?location=India"} 

68,000+ Tech jobs in India
 Agree & Join LinkedIn
 By clicking Continue, you agree to LinkedIn’s User Agreement, Privacy Policy, and Cookie Policy.
 Skip to main content
LinkedIn
 Tech in India
Expand search
This button displays the currently selected search type. When expanded it provides a list of search options that will switch the search inputs to match the current selection. Jobs
 People
 Learning
Clear text
Clear text
Clear text
Clear text
Clear text
 Join now
 Sign in
 Any time
 

In [ ]:
from IPython.display import Markdown
Markdown(result)

Here is the final answer in a structured format:

**Company** | **Job Title** | **Location** | **Hiring Status**
---------|------------|-----------|----------------
Infosys | IT Testing | Bengaluru East, Karnataka, India | Actively Hiring (1 month ago)
Infosys | IT Testing | Bengaluru East, Karnataka, India | Actively Hiring (2 months ago)
Infosys | IT Testing | Pune, Maharashtra, India | Actively Hiring (1 month ago)
Deloitte | Business Analyst | Mumbai, Maharashtra, India | Actively Hiring (1 day ago)
Infosys | IT Testing | Bengaluru East, Karnataka, India | Actively Hiring (1 month ago)
Infosys | IT Testing | Bengaluru East, Karnataka, India | Actively Hiring (2 months ago)
Circle K | Associate Data Analyst | Gurugram, Haryana, India | Actively Hiring (5 days ago)
Infosys | IT Testing | Hyderabad, Telangana, India | Actively Hiring (2 months ago)
Deloitte | Business Analyst | Mumbai, Maharashtra, India | Actively Hiring (1 day ago)
Infosys | IT Testing | Bengaluru East, Karnataka, India | Actively Hiring (1 month ago)
Tech Mahindra | Business Analyst | Pune, Maharashtra, India | Actively Hiring (3 weeks ago)
Infosys | IT Testing | Bengaluru East, Karnataka, India | Actively Hiring (2 months ago)
airtel | Non-Tech Recruiter-Off Role | Gurugram, Haryana, India | Actively Hiring (1 week ago)
Infosys | IT Consulting | Bengaluru East, Karnataka, India | Actively Hiring (1 month ago)
Infosys | IT Testing | Bengaluru East, Karnataka, India | Actively Hiring (2 months ago)
Tech Mahindra | SAP MM Consultant | Vishakhapatnam, Andhra Pradesh, India | Actively Hiring (4 weeks ago)
Infosys | IT Consulting | Bengaluru East, Karnataka, India | Actively Hiring (1 month ago)
Britannia Industries Limited | Production Officer | Ranjangaon, India | Actively Hiring (3 days ago)
Infosys | IT Consulting | Bengaluru East, Karnataka, India | Actively Hiring (1 month ago)
Infosys | IT Testing | Coimbatore, Tamil Nadu, India | Actively Hiring (2 months ago)
Infosys | IT Consulting | Bengaluru East, Karnataka, India | Actively Hiring (1 month ago)
Infosys | IT Consulting | Bengaluru East, Karnataka, India | Actively Hiring (2 months ago)
Infosys | IT Testing | Bengaluru East, Karnataka, India | Actively Hiring (2 months ago)
Infosys | IT Consulting | Bengaluru East, Karnataka, India | Actively Hiring (1 month ago)
Infosys | IT Consulting | Bengaluru East, Karnataka, India | Actively Hiring (1 month ago)
Infosys | IT Consulting | Bengaluru East, Karnataka, India | Actively Hiring (2 months ago)
Infosys | IT Consulting | Bengaluru East, Karnataka, India | Actively Hiring (2 months ago)
Flipkart | Business Analyst | Bengaluru, Karnataka, India | Actively Hiring (5 days ago)
Infosys | IT Consulting | Bengaluru East, Karnataka, India | Actively Hiring (1 month ago)
Infosys | IT Consulting | Bengaluru East, Karnataka, India | Actively Hiring (1 month ago)
Tech Mahindra | Business Analyst | Hyderabad, Telangana, India | Actively Hiring (1 day ago)
Infosys | IT Testing | Trivandrum, Kerala, India | Actively Hiring (2 months ago)
Infosys | IT Testing | Pune, Maharashtra, India | Actively Hiring (1 month ago)

Note: The above table only shows a subset of the total jobs available on the LinkedIn website.